In [128]:
import torch
from torch import nn
import numpy as np

In [129]:
import random
class myConv1d():
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int = 1) -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

        # init parameters
        self.parameters = torch.randn((out_channels, in_channels, kernel_size))

        # init bias
        self.bias = torch.randn((out_channels))

    def __call__(self, input: torch.Tensor) -> torch.Tensor:
        calculated = torch.zeros((self.out_channels, input.size(1) - self.kernel_size + 1))
        for i_out in range(self.out_channels):
            for i_in in range(self.in_channels):
                for i_w in range(calculated.shape[1]):
                    # print(i_out, i_in, i_w)
                    calculated[i_out][i_w] += torch.dot(self.parameters[i_out, i_in], input[i_in, i_w:i_w + self.kernel_size])
        calculated += self.bias.reshape((self.out_channels, 1))
        return calculated

In [130]:
IN_CHANNEL = 1
OUT_CHANNEL = 2
KERNEL_SIZE = 3
HEIGHT = 3
WIDTH = 3
LENGTH = 4
t = torch.arange(IN_CHANNEL * LENGTH, dtype=torch.float)
t.resize_(IN_CHANNEL, LENGTH)
t

tensor([[0., 1., 2., 3.]])

In [131]:
d1 = nn.Conv1d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE, dtype=torch.float)
d2 = myConv1d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE)
d2.parameters = d1._parameters['weight'].data
d2.bias = d1._parameters['bias'].data
d2(t), d1(t)

(tensor([[0.5475, 1.1547],
         [0.9087, 1.6138]]),
 tensor([[0.5475, 1.1547],
         [0.9087, 1.6138]], grad_fn=<SqueezeBackward1>))

In [132]:
import random
class myConv2d():
    def __init__(self, in_channels: int, out_channels: int, kernel_size: tuple[int, int], stride: int = 1) -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

        # init parameters
        self.parameters = torch.randn((self.out_channels, self.in_channels, *kernel_size))

        # init bias
        self.bias = torch.randn((self.out_channels))

    def __call__(self, input: torch.Tensor) -> torch.Tensor:
        calculated = torch.zeros(self.out_channels, input.shape[1] - self.kernel_size[0] + 1, input.shape[2] - self.kernel_size[1] + 1)
        for i_out in range(calculated.shape[0]):
            for i_in in range(self.in_channels):
                for i_w in range(calculated.shape[1]):
                    for i_h in range(calculated.shape[2]):
                        calculated[i_out][i_h][i_w] += torch.sum(self.parameters[i_out, i_in] * input[i_in, i_w:i_w + self.kernel_size[0], i_h:i_h + self.kernel_size[1]])
        calculated += self.bias.reshape((self.out_channels, 1, 1))
        return calculated

In [133]:
IN_CHANNEL = 2
OUT_CHANNEL = 2
KERNEL_SIZE = (3, 3)
HEIGHT = 4
WIDTH = 4
t1 = torch.arange(IN_CHANNEL * WIDTH * HEIGHT, dtype=torch.float)
t1.resize_(IN_CHANNEL, WIDTH, HEIGHT)
t1

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]],

        [[16., 17., 18., 19.],
         [20., 21., 22., 23.],
         [24., 25., 26., 27.],
         [28., 29., 30., 31.]]])

In [134]:
d3 = nn.Conv2d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE)
d4 = myConv2d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE)
d4.parameters = d3._parameters['weight'].data
d4.bias = d3._parameters['bias'].data
d3(t1), d4(t1)

(tensor([[[13.9311, 14.7224],
          [17.0962, 17.8874]],
 
         [[ 2.0578,  1.8111],
          [ 1.0712,  0.8245]]], grad_fn=<SqueezeBackward1>),
 tensor([[[13.9311, 17.0962],
          [14.7224, 17.8874]],
 
         [[ 2.0578,  1.0712],
          [ 1.8111,  0.8245]]]))

In [135]:
X = torch.arange(8).reshape((1, 8))
X.sum()

tensor(28)